In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
import sys
sys.path.append('/content/drive/My Drive/APT/newest/TAR-project-master')

In [0]:
pip install num2words

     |████████████████████████████████| 102kB 3.9MB/s 


In [0]:
pip install langdetect

     |████████████████████████████████| 983kB 4.6MB/s 
  Created wheel for langdetect: filename=langdetect-1.0.8-cp36-none-any.whl size=993193 sha256=413dc10804fc5088e3c63bd64ce08977190e6d0194c83a5f40c08bc6eb7587cb
  Stored in directory: /root/.cache/pip/wheels/8d/b3/aa/6d99de9f3841d7d3d40a60ea06e6d669e8e5012e6c8b947a57
Successfully built langdetect


In [0]:
pip install -U sentence-transformers

     |████████████████████████████████| 61kB 3.0MB/s 
     |████████████████████████████████| 645kB 13.8MB/s 
     |████████████████████████████████| 1.1MB 29.8MB/s 
     |████████████████████████████████| 3.8MB 44.2MB/s 
     |████████████████████████████████| 890kB 38.3MB/s 
  Created wheel for sentence-transformers: filename=sentence_transformers-0.2.6.1-cp36-none-any.whl size=74031 sha256=5293f748cf2e8c7bc097f98245fb374650d9cc4359897e2ff1108fbd08ba60f4
  Stored in directory: /root/.cache/pip/wheels/d7/fa/17/2b081a8cd8b0a86753fb0e9826b3cc19f0207062c0b2da7008
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893260 sha256=b49964856fe2729751a123c5d6405bd0ce7c3526db265ac664e7762079eedc9f
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sentence-transformers sacremoses


In [0]:
import pandas as pd
import numpy as np
import nltk

nltk.download('stopwords')
nltk.download('punkt')

from dataset.preprocessing.preprocessing import word_stem

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [0]:
abstracts_path = '/content/drive/My Drive/APT/temp_data/cvite_data/body_texts.csv'
abstracts = pd.read_csv(abstracts_path)
abstracts.head()

,paper_id,section,text,preprocessed_text
0,0001418189999fea7f7cbe3e82703d71c85a6fe5,Introduction,Feline infectious peritonitis (FIP) is a fatal...,felin infecti periton fip fatal chronic diseas...
1,00016663c74157a66b4d509d5c4edffd5391bbe0,Thoughts on other viruses,The data described also illustrated that curre...,the data describ also illustr current microbio...
2,00039b94e6cb7609ecbddee1755314bcfeb77faa,INTRODUCTION,Severe acute respiratory syndrome (SARS) is a ...,sever acut respiratori syndrom sar recent emer...
3,00039b94e6cb7609ecbddee1755314bcfeb77faa,Patients,Our study included 20 consecutive adult patien...,our studi includ twenti consecut adult patient...
4,0005d253951fedc237715a37db147032eea28912,NaN,Even in the case of successful antiviral thera...,even case success antivir therapi sword damocl...


In [0]:
dupl = abstracts[abstracts.duplicated(subset='preprocessed_text')]

In [0]:
dupl.head().style.set_properties(**{'font-size': '9pt'})

## Queries

In [0]:
queries2 = [
           'What is known about transmission, incubation, and environmental stability?',
           'What do we know about natural history, transmission, and diagnostics for the virus?',
           'What have we learned about infection prevention and control?',
           "incubation period of COVID-19",
           "Does smoking increase risks when having covid19?",
           "mortality rate of covid19?",
           "pregnancy complications covid19?"
]

queries = [
           'Does smoking affect severness of covid19 infection?',
          'How are pregnant women affected by covid19?',
          'Co-morbidities affecting covid19',
          'What is the fatality of covid19?',
          'What is the basic reproductive number of covid19?',
          'What is the incubation period of covid19?',
          'Most common ways of transmission of covid19',
          'Are enviromental factors affecting transmission of covid19?'
]


color_list = ['black', 'red', 'blue', 'green']
colors = {}
for i in range(len(queries)):
  colors[queries[i]] = color_list[i % len(color_list)]


def run_queries(query_engine, queries, k=5):
  results = query_engine.run_query(queries[0], k)
  if len(queries) > 1:
    for q in queries[1:]:
      results = pd.concat([results, query_engine.run_query(q, k)])
  return results

def color_rows(val):
    col = colors[val['query']]
    return ['color: %s' % col]*len(val)

In [0]:
#print(colors)

{'incubation period of COVID-19': 'black', 'Does smoking increase risks when having covid19?': 'red', 'mortality rate of covid19?': 'blue', 'pregnancy complications covid19?': 'black'}


## TF-IDF

In [0]:
from nltk.corpus import stopwords
from query_model.query_engines import BOWQueryEngine
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer

In [0]:
stop_words = set(stopwords.words('english'))
cv = CountVectorizer(stop_words=stop_words)
transformer = TfidfTransformer(smooth_idf=True, use_idf=True)
query_engine = BOWQueryEngine(cv, transformer)
query_engine.fit(abstracts)

In [0]:
# If you want to load already trained model
# query_engine = BOWQueryEngine.load('/content/drive/My Drive/APT/trained_models/tf_idf.dat')

In [0]:
#query_engine.save('/content/drive/My Drive/APT/trained_models/', 'tf_idf')

Writing object to /content/drive/My Drive/APT/trained_models/tf_idf.dat


In [0]:
res = run_queries(query_engine, queries)
res.reset_index(drop=True)[['query', 'text', 'sim']].style.set_properties(**{'font-size': '9pt'}).apply(color_rows, axis=1)

## BM25

In [0]:
from nltk.corpus import stopwords
from query_model.query_engines import BOWQueryEngine
from query_model.transformers.bm25 import BM25Transformer

In [0]:
stop_words = set(stopwords.words('english'))
cv = CountVectorizer(stop_words=stop_words)
transformer = BM25Transformer()
query_engine = BOWQueryEngine(cv, transformer)
query_engine.fit(abstracts)

In [0]:
# If you want to load already trained model
# query_engine = BOWQueryEngine.load('/content/drive/My Drive/APT/trained_models/bm25.dat')

In [0]:
#query_engine.save('/content/drive/My Drive/APT/trained_models/', 'bm25')

Writing object to /content/drive/My Drive/APT/trained_models/bm25.dat


In [0]:
res = run_queries(query_engine, queries)
res.reset_index(drop=True)[['query', 'text', 'sim']].style.set_properties(**{'font-size': '9pt'}).apply(color_rows, axis=1)

## Word2Vec

In [0]:
from query_model.query_engines import W2VQueryEngine

In [0]:
params = {
    'min_count': 5,
    'size': 300, 
    'workers': 3, 
    'window': 3,
    'sg': 0,
    'hs': 0,
    'negative': 5
    }
query_engine = W2VQueryEngine(params)
query_engine.fit(abstracts)

In [0]:
# If you want to load already trained model
query_engine = W2VQueryEngine.load('/content/drive/My Drive/APT/trained_models/w2v_body_texts.dat')

In [0]:
query_engine.save('/content/drive/My Drive/APT/trained_models/', 'w2v_body_texts2')

Writing object to /content/drive/My Drive/APT/trained_models/w2v_body_texts.dat


In [0]:
res = run_queries(query_engine, queries)
res.reset_index(drop=True)[['query', 'text', 'sim']].style.set_properties(**{'font-size': '9pt'}).apply(color_rows, axis=1)

## Doc2Vec

In [0]:
from query_model.query_engines import D2VQueryEngine

In [0]:
params = { 
    'vector_size': 300,
    'min_count': 5, 
    'negative': 5, 
    'hs': 0, 
    'sample': 1e-5,
    'epochs': 20,
    'window': 15
    }
query_engine = D2VQueryEngine(params)
query_engine.fit(abstracts)

In [0]:
# If you want to load already trained model
query_engine = D2VQueryEngine.load('/content/drive/My Drive/APT/trained_models/d2v_body_texts.dat')

In [0]:
query_engine.save('/content/drive/My Drive/APT/trained_models/', 'd2v_body_texts')
#query_engine.save('/content/drive/My Drive/APT/trained_models/', 'd2v15000')

Writing object to /content/drive/My Drive/APT/trained_models/d2v15000.dat


In [0]:
res = run_queries(query_engine, queries)
res.reset_index(drop=True)[['query', 'text', 'sim']].style.set_properties(**{'font-size': '9pt'}).apply(color_rows, axis=1)

100%|██████████| 1/1 [00:00<00:00, 260.68it/s]


,query,text,sim
0,Does smoking affect severness of covid19 infection?,"Social isolation and loneliness are both important modifiable factors impacting mental and physical health in older adults. For example, in meta-analyses, social isolation and loneliness were associated with a 50% increased risk of developing dementia, 2 an approximately 30% increased risk of incident coronary artery disease or stroke, 3 and a 26% increased risk of all-cause mortality. 4 Of special concern to the geriatric psychiatry community are the mental health implications for older adults. These markers of social disconnection increase the likelihood of developing or worsening late life depression and anxiety and, reciprocally, these psychiatric conditions also predispose to social disconnection. 5, 6 Though late adulthood is often a period of socialemotional well-being, it also poses an increasing risk of loneliness in the setting of age-related factors such as the loss of close relationships and the contraction of social networks, sensory impairments (especially hearing loss), chronic health conditions, mobility limitations, and other physical and cognitive declines. A nationally representative German study of over 16,000 adolescents and adults found the lowest prevalence of loneliness among individuals in the sixth to seventh decades of life, yet striking elevations in loneliness among those in the eighth decade and beyond. 7 For these reasons, effective interventions for social disconnection, especially loneliness, are much needed for older people.",0.894116
1,Does smoking affect severness of covid19 infection?,"The usage of humanized mice in the development of vaccines targeting human diseases including EBV, HIV-1, dengue virus, infl uenza virus, severe acute respiratory syndrome (SARS) corona virus, and carcino-embryonic antigen (CEA) has obtained outstanding achievements during the past decade, while the introduction of HLA transgenic immunodefi cient mice further accelerated the advancement in this fi eld [ 171 -173 ] . With the improvement of immune cell population reconstitution, more and more novel vaccination protocol will be carried out in humanized mice.",0.892342
2,Does smoking affect severness of covid19 infection?,"Some noncovalent docking hits act as covalent artifacts, often via a reactive center present on the docked molecule or on an impurity. A rapid counterscreen for irreversible inhibition is to incubate the target protein and the hit at 5× their apparent IC 50 and then dilute them 10fold (other IC 50 ratios may of course be chosen). If inhibition is rapidly reversible, the inhibition on dilution should drop to about 33% of full inhibition at this ratio. If dilution changes the inhibition little, it supports covalent activity. Legitimate, slow off-rate inhibition is another alternative, but such molecules are rare among initial screening hits. This experiment will only work for soluble proteins, but related experiments to measure off-rate may be adapted for membrane proteins.",0.892009
3,Does smoking affect severness of covid19 infection?,"Although we have been carefully removing the drape after successful intubation, we feel that Brown et al 1 and other authors make an excellent point that the sheet can be left in place for the duration of surgery, and the patient can be subsequently extubated under the drape, shielding anesthesia providers and other operating room personnel when the endotracheal tube is removed. 4 During these trying times, it is encouraging to see how health care professionals over the globe are readily sharing clinical insights, and we hope that our experiences with a simple modification to the barrier sheet method may help others improve their success rate of initial intubation while still providing protection to anesthesia professionals during the COVID-19 pandemic.",0.890478
4,Does smoking affect severness of covid19 infection?,"The regression results on SF-12 show that older workers enjoyed better menta

## BERT

In [0]:
from sentence_transformers import SentenceTransformer
from query_model.query_engines import BERTQueryEngine

In [0]:
import torch
torch.cuda.get_device_name(0)

'Tesla P100-PCIE-16GB'

In [0]:
query_engine2 = BERTQueryEngine()
query_engine2.fit(abstracts, text_column='text')

In [0]:
# If you want to load already trained model
#query_engine = BERTQueryEngine.load('/content/drive/My Drive/APT/trained_models/BERT.dat')
query_engine = BERTQueryEngine.load('/content/drive/My Drive/APT/trained_models/BERT_on_texts.dat')

In [0]:
#query_engine2.save('/content/drive/My Drive/APT/trained_models/', 'BERT_on_texts')

Writing object to /content/drive/My Drive/APT/trained_models/BERT_on_texts.dat


In [0]:
res = run_queries(query_engine, queries)
res.reset_index(drop=True)[['query', 'text', 'sim']].style.set_properties(**{'font-size': '9pt'}).apply(color_rows, axis=1)